In [ ]:
# Test runtime
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  6 12:54:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Mount drive immediately
from google.colab import drive
drive.mount('/content/drive')

# Model path
# - Mårten
model_base = "/content/drive/MyDrive/id2223-lab"
model_path = model_base + "/whisper-small-sv"

# - Maria
# model_base = "/content/drive/MyDrive/id2223-lab"
# model_path = model_base + "/whisper-small-sv"

Mounted at /content/drive


In [ ]:
# Check if drive is mounted
import os
if not os.path.exists(model_base):
  raise Exception("Drive does not seem to be mounted!")

In [ ]:
# Install updated libaries
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install accelerate==0.23.0
!pip install datasets transformers[sentencepiece]

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3oxm2py8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-3oxm2py8
  Resolved https://github.com/huggingface/transformers to commit d6392482bd8af0d28903b5531ed04bd7273931a3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8148049 sha256=81192ecb344c2da641c636c6637ec4cf2393c96d690ba523bec3daae2d08ff24
  Stored in directory: /tmp/pip-ephem-wheel-cache-gbvyach3/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Log in to HF
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load feature extractor from pre-trained checkpoint
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

# Load WhisperTokenizer
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")

# Create a WhisperProcessor
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [ ]:
def load_common_voice_remote(dataset_repo, dataset_version):
  # Load our dataset, use latest version
  from datasets import load_dataset, DatasetDict

  common_voice = DatasetDict()

  common_voice["train"] = load_dataset(dataset_repo, dataset_version, split="train+validation", use_auth_token=True)
  common_voice["test"] = load_dataset(dataset_repo, dataset_version, split="test", use_auth_token=True)

  print(common_voice)

  # Remove features we don't need (variant is new and can be removed)
  common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])
  print(common_voice)

  # Check first example in data
  print(common_voice["train"][0])

  # Downsample to 16 kHz
  from datasets import Audio
  common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

  # Check the first data sample again
  print(common_voice["train"][0])

  # Function to prepare the dataset
  def prepare_dataset(batch):
      # load and resample audio data from 48 to 16kHz
      audio = batch["audio"]

      # compute log-Mel input features from input audio array
      batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

      # encode target text to label ids
      batch["labels"] = tokenizer(batch["sentence"]).input_ids
      return batch

  # Prepare the full dataset
  common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

  # Push to hub so we don't have to prepare/pre-process the dataset the next time
  common_voice.push_to_hub("GroupSix/common-voice-sv")

  return common_voice

def load_common_voice_cache(dataset_repo):
  # Load our dataset from cache
  from datasets import load_dataset
  common_voice = load_dataset(dataset_repo)
  return common_voice

In [ ]:
# Load common voice from cache if it exists
import os
if os.path.exists(model_path):
  common_voice = load_common_voice_cache("GroupSix/common-voice-sv")

else:
  common_voice = load_common_voice_remote("mozilla-foundation/common_voice_13_0", "sv-SE")

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/12521 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5120 [00:00<?, ? examples/s]

In [ ]:
# Data collator
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

# Initialize data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# Use WER metric for evaluation
import evaluate
metric = evaluate.load("wer")

# Define function to compute metrics based on WER
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
# Load a Pre-Trained Checkpoint
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
# Override arguments
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False # disable use cache during gradient checkpointing, disable during inference
# https://discuss.huggingface.co/t/why-is-use-cache-incompatible-with-gradient-checkpointing/18811/5

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/id2223-lab/whisper-small-sv",            # Drive dir
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,                                              # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,                                                         # Maybe we should tune this then?
    warmup_steps=500,
    max_steps=2000,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':True},                       # Added to remove error
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    # push_to_hub=True,                                                         # Don't push to hub, we use drive for checkpoints
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor
)

In [ ]:
# Resume training if at least "checkpoint-1000" is saved, otherwise start from beginning (and overwrite)
if os.path.exists(model_path + "/checkpoint-1000"):
  trainer.train(resume_from_checkpoint=True)
else:
  # Save once before training
  processor.save_pretrained(training_args.output_dir)
  trainer.train()

There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


Step,Training Loss,Validation Loss


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


In [ ]:
# Push model
trainer.save_model()
trainer.push_to_hub("GroupSix/whisper-small-sv")
tokenizer.push_to_hub("GroupSix/whisper-small-sv")

CommitInfo(commit_url='https://huggingface.co/GroupSix/whisper-small-sv/commit/b2003f4486fb74e536224539ac41665b42ead880', commit_message='Upload tokenizer', commit_description='', oid='b2003f4486fb74e536224539ac41665b42ead880', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub("GroupSix/whisper-small-sv")

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/GroupSix/whisper-small-sv/commit/b7cb295232764490a6a05b640586bb389545b6a8', commit_message='Upload WhisperForConditionalGeneration', commit_description='', oid='b7cb295232764490a6a05b640586bb389545b6a8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print(model)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f